In [1]:
import psycopg2
import csv

In [5]:
try:
    conn = psycopg2.connect("host=localhost dbname=ETL_PROJECT user=testuser password=password")
    cur = conn.cursor()
    #Load OUTCOMES_FACTORS_SUBRANKINGS_STG
    with open("./Resources/2019 County Health Rankings North Carolina Data - v1_0 (2).txt", 'r') as f1:
        # Skip the first three header rows.
        next(f1)
        next(f1)
        next(f1)
        #Tab or | delimited is the way to go.  Never use comma delimited files.
        cur.copy_from(f1, 'OUTCOMES_FACTORS_SUBRANKINGS_STG', sep='\t')
        conn.commit()
    #Load TRI_FACILITIES_ALL_STG
    with open("./Resources/tri_facilities_all.txt", 'r') as f2:
        next(f2)
        #This file is a perfect example as to why you never use comma delimited files.
        cur.copy_from(f2, 'TRI_FACILITIES_ALL_STG', sep='\t')
        conn.commit() 

    #Load FIPS_DIMENSION
    sql = """
    INSERT INTO fips_dimension
    SELECT DISTINCT
             fips
           , state
           , county
    FROM
             OUTCOMES_FACTORS_SUBRANKINGS_STG
    ;"""
    cur.execute(sql)
    conn.commit()

    #Load OUTCOMES_FACTORS_SUBRANKINGS_FACT
    sql = """
    INSERT INTO OUTCOMES_FACTORS_SUBRANKINGS_FACT
    SELECT DISTINCT 
           A.fips
         , A.lngth_of_lf_z_2019_score
         , A.lngth_of_lf_2019_rank
         , A.qlty_of_lf_z_2019_score
         , A.qlty_of_lf_2019_rank
         , A.hlth_of_bhvr_z_2019_score
         , A.hlth_of_bhvr_2019_rank
         , A.clncl_cr_z_2019_score
         , A.clncl_cr_2019_rank
         , A.soc_eco_fctrs_z_2019_score
         , A.soc_eco_fctrs_2019_rank
         , A.phys_en_z_2019_score
         , A.phys_en_2019_rank
         ,B.TOT_RLS_CUMUL_2016
         ,B.TOT_RLS_2016
         ,B.TOT_RLS_SITE_COUNTS
    FROM public.outcomes_factors_subrankings_stg A
    LEFT JOIN 
        (
        SELECT CAST(column8 AS INTEGER) AS FIPS
             , SUM(CAST(column12 AS DECIMAL(20,5))) AS TOT_RLS_CUMUL_2016
             , SUM(CASE WHEN CAST(column13 AS DECIMAL(20,5)) = -1
               THEN NULL
               ELSE CAST(column13 AS DECIMAL(20,5))
               END) AS TOT_RLS_2016
            ,COUNT(*) AS TOT_RLS_SITE_COUNTS
        FROM public.tri_facilities_all_stg
        GROUP BY CAST(column8 AS INTEGER)
        ) B ON(A.FIPS = B.FIPS)
    ;"""
    cur.execute(sql)
    conn.commit()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()